# Defi Wind - Results Download

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [defi-results](https://github.com/Logiqx/defi-results) and is distributed under the terms of the GNU General Public License.

defi-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import json

import urllib.request

import unicodedata
import re

In [2]:
projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

In [3]:
CONFIG_DIR = 'config'

EVENTS_CONFIG = 'events.json'

In [4]:
if __name__ == '__main__':
    filename = os.path.join(projdir, CONFIG_DIR, EVENTS_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        events = json.loads(jsonTxt)

In [5]:
def slugify(value):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [6]:
DATA_DIR = 'data'
JSON_DIR = 'json'
CSV_DIR = 'csv'

EVENT_NAME = 'name'
EVENT_FIELDS = 'fields'
EVENT_RACES = 'races'

RACE_NAME = 'name'
RACE_URL = 'url'

RESULTS_LIST = 'list'
RESULTS_FIELDS = 'Fields'
RESULTS_DATA = 'data'

FIELD_LABEL = 'Label'

In [7]:
def getFieldPositions(event, results):
    '''Get field positions from results'''
    
    positions = []

    # Nationality is sometimes blank
    labels = [field[FIELD_LABEL] or 'Nat.' for field in results[RESULTS_LIST][RESULTS_FIELDS]]

    # Determine index of all fields
    for field, aliases in event[EVENT_FIELDS].items():
        position = None

        for alias in aliases:
            if alias in labels:
                position = labels.index(alias)
                break
                
        positions.append(position)

    return positions


def getData(event, results, fieldPositions):
    '''Get data from results'''
      
    data = []
    
    for result in results[RESULTS_DATA]:
        # Defi Wind 2022 (run 5) has tally as the first field and place as the second field
        if result[0] != result[2].replace('#', ''):
            offset = 2
        else:
            offset = 1

        record = []
        for idx in fieldPositions:
            if idx is not None:
                record.append(result[idx + offset])
            else:
                record.append('')

        data.append(record)

    print(data[:1])
    return data
        

def processEvent(event):
    '''Process single event'''

    jsonPath = os.path.join(projdir, DATA_DIR, JSON_DIR, slugify(event[EVENT_NAME]))    
    
    if not os.path.exists(jsonPath):
        os.makedirs(jsonPath)
            
    for race in event[EVENT_RACES]:
        slug = slugify(race[RACE_NAME]) + '.json'
        fn = os.path.join(jsonPath, slug)
        
        print(f'Downloading {event[EVENT_NAME]} - {race[RACE_NAME]}...')

        with urllib.request.urlopen(race[RACE_URL]) as f:
            jsonTxt = f.read().decode('utf-8')
            results = json.loads(jsonTxt)
            jsonTxt = json.dumps(results, indent=2)

        fieldPositions = getFieldPositions(event, results)
        data = getData(event, results, fieldPositions)

        #with open(fn, 'w') as f:
            #f.write(jsonTxt)


for event in events:
    processEvent(event)

print('All done!')

[['1.', '192', 'ALBEAU, Antoine', '[img:flags/FR.gif]', 'M (1)', '40-49M (1)', '00:33:00', '0,7']]
[['1.', '465', 'GOYARD, Nicolas', '[img:flags/FR.gif]', 'M (1)', '20-29M (1)', '00:32:11', '0,7']]
[['1.', '465', 'GOYARD, Nicolas', '[img:flags/FR.gif]', 'M (1)', '20-29M (1)', '00:31:22', '0,7']]
[['1.', '465', 'GOYARD, Nicolas', '[img:flags/FR.gif]', 'M (1)', '20-29M (1)', '00:34:24', '0,7']]
[['1.', '465', 'GOYARD, Nicolas', '[img:flags/FR.gif]', 'M (1)', '20-29M (1)', '', '0,7']]
[['1.', '#465', 'GOYARD, NICOLAS', '[img:https://chronoconsult.fr/resultat/flags/fr_black.png]', 'M (1)', 'Foil (1)', '20-39M (1)', '00:32:19']]
[['1.', '#465', 'GOYARD, NICOLAS', '[img:https://chronoconsult.fr/resultat/flags/fr_black.png]', 'M (1)', 'Foil (1)', '20-39M (1)', '00:30:35']]
[['1.', '#465', 'GOYARD, NICOLAS', '[img:flags/FR.gif]', 'M (1)', 'Foil (1)', '20-39M (1)', '00:30:50']]
[['1.', '#37', 'SØE, JOHAN', '[img:flags/DK.gif]', 'M (1)', 'Libre (1)', 'U20M (1)', '00:31:18']]
All done!
